# Tasks
## Task 1 : Binary Representations

I will do this using several functions rotate left, rotate right  a choose function and an majority function. Let's start with the roate left function I want to rotate the bits in a 32 bit unsigned integer too n places n being anything so lets do this one first. 


In [196]:
d = 0xe3


In [197]:
print(d)

227


In [198]:
bin(d)

'0b11100011'

In [199]:
bin(d <<3)

'0b11100011000'

In [200]:
bin(d >> 5)

'0b111'

In [201]:
bin((d << 3) | (d >> 5))

'0b11100011111'

In [202]:
0xb << 3

88

In [203]:
import numpy

int_bits = 32

x=2
n=42

def rotl(n ,x):

    return (n << x)|(n >> (int_bits - x))




In [204]:
print("Left Rotation of",n,"by"
      ,x,"is",end=" ")
print(rotl(n, x))

Left Rotation of 42 by 2 is 168


In [205]:
def rotr(n ,x):

    return (n >> x)|(n << (int_bits - x)) & 0xFFFFFFFF

In [206]:
print("Right Rotation of",n,"by"
      ,x,"is",end=" ")
print(rotr(n, x))

Right Rotation of 42 by 2 is 2147483658
